We will start with cleaning the datasets. As we did in data exploration note book. 

In [1]:
! pip install tldextract

     |████████████████████████████████| 87 kB 956 kB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
from tldextract import extract
import re,sys,pickle

In [3]:
#nltk libraries
import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
train_df = pd.read_csv('../input/zsdataset/train.csv',encoding= 'ISO-8859-1')
test_df = pd.read_csv('../input/zsdataset/test.csv',encoding= 'ISO-8859-1')

As we did in data exploration note book we will clean our dataset. 

In [6]:
#extracts the url from the link, we will extract hostname only
def extract_url(x):
    tsd, td, tsu = extract(x) # prints abc, hostname, com
    return td
def classes_def(x):
    if x ==  "FACEBOOK":
        return "Facebook"
    elif x == 'FORUMS':
        return 'Forums'
    elif x == 'BLOG':
        return 'Blog'
    elif x == 'YOUTUBE':
        return 'Youtube'
    else:
        return 'Facebook'

In [7]:
train_df['Source']=train_df['Source'].apply(lambda x:classes_def(x))
test_df['Source']=test_df['Source'].apply(lambda x:classes_def(x))
train_df["Host"].fillna(train_df["Link"], inplace=True)
test_df["Host"].fillna(test_df["Link"], inplace=True)
train_df['Host']=train_df['Host'].apply(lambda x:extract_url(x))
test_df['Host']=test_df['Host'].apply(lambda x:extract_url(x))
train_df.loc[train_df.Host == '' , 'Host'] = 'youtube'

Let's drop columns which are not useful, we will be dropping Link, time(GMT), as it gives same information as Host and Time(ET) respectively. Also, Title has maximum 20% Null values. It can be affect the training of dataset. So we will drop it too

In [8]:
train_df = train_df.drop(['Link','time(GMT)','Title'],axis= 1)
test_df = test_df.drop(['Link','time(GMT)','Title'],axis= 1)

At last, make date and time columns, in date-time format.

In [9]:
train_df['Date(ET)'] = pd.to_datetime(train_df['Date(ET)'],errors='coerce').dt.date
test_df['Date(ET)'] = pd.to_datetime(test_df['Date(ET)'],errors='coerce').dt.date

In [10]:
train_df['Time(ET)'] = pd.to_datetime(train_df['Time(ET)'],errors='coerce').dt.time
test_df['Time(ET)'] = pd.to_datetime(test_df['Time(ET)'],errors='coerce').dt.time

Now we will have, NaT elements in date and time. We won't remove them now as we are not adding this features while generating model. But, later if we use these features then, we will remove NaT rows from dataset. 

Also remove Unnamed:9 and Index columns from test datasets

In [11]:
test_df.drop(columns = ['Index','Unnamed: 9'],inplace = True)

Also, there is one row in TRANS_CONV_TEXT is having null, we will drop that row

In [12]:
train_df = train_df[train_df['TRANS_CONV_TEXT'].notna()]
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1156 entries, 0 to 1156
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Source           1156 non-null   object
 1   Host             1156 non-null   object
 2   Date(ET)         1156 non-null   object
 3   Time(ET)         1084 non-null   object
 4   TRANS_CONV_TEXT  1156 non-null   object
 5   Patient_Tag      1156 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 63.2+ KB


In [13]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 571 entries, 0 to 570
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Source           571 non-null    object
 1   Host             571 non-null    object
 2   Date(ET)         570 non-null    object
 3   Time(ET)         532 non-null    object
 4   TRANS_CONV_TEXT  571 non-null    object
dtypes: object(5)
memory usage: 22.4+ KB


The last thing is data preprocessing. We will clean text and remove, all numbers and punctuations from the text data, TRANS_CONV_TEXT

In [14]:
def clean_text(x):
    normalizedsentense = x.lower()
    text = re.sub(r"[^a-z']+", ' ', normalizedsentense)
    return text

In [15]:
train_df['TRANS_CONV_TEXT']=train_df['TRANS_CONV_TEXT'].apply(lambda x:clean_text(x))
test_df['TRANS_CONV_TEXT']=test_df['TRANS_CONV_TEXT'].apply(lambda x:clean_text(x))

In [16]:
train_df.head()

,Source,Host,Date(ET),Time(ET),TRANS_CONV_TEXT,Patient_Tag
0,Forums,cafepharma,2016-06-15,13:58:00,i don't disagree with you in principle i'm jus...,0
1,Forums,patient,2016-05-07,NaT,i am always dizzy i get dizzy standing up so i...,1
2,Blog,abcnewsradioonline,2016-04-14,15:00:38,axelle bauer griffin filmmagic new york queen ...,0
3,Forums,cancer-forums,2016-06-18,20:46:00,i am and i have been throwing up for about a y...,1
4,Forums,diyaudio,2016-06-15,03:26:00,quote originally posted by boyan silyavski wak...,0


In [17]:
test_df.head()

,Source,Host,Date(ET),Time(ET),TRANS_CONV_TEXT
0,Blog,uhmagonline,2016-07-30,00:41:23,baby slice the son of the late kimbo slice has...
1,Forums,yuku,2016-06-20,00:07:30,p font face sans serif size i have had both s...
2,Blog,blogspot,2016-06-15,15:44:00,previously sodium glucose cotransporter sglt i...
3,Forums,healthboards,2016-07-17,19:41:00,hello i suffer from congestive heart failure d...
4,Blog,sciencecodex,2016-04-04,15:30:45,a daily dose of vitamin d improves heart funct...


Now it looks good. We will start modeling.

### Let's create a model with Logistic regression and bag of words and 5 kfolds

* Bag of Words (BOW) is a method to extract features from text documents. These features can be used for training machine learning algorithms. It creates a vocabulary of all the unique words occurring in all the documents in the training set. 
*  logistic regression is one of the most important analytic tools in the social and natural sciences. In natural language processing, logistic regression is the baseline supervised machine learning algorithm for classification, and also has a very close relationship with neural networks.

In [62]:
df = train_df.copy()
df_test = test_df.copy()

In [67]:
from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


if __name__ =="__main__":
    #create a new column with and fill it with -1
    df['kfold'] = -1
    #randomize the data
    df = df.sample(frac = 1).reset_index(drop =True)
    y = df.Patient_Tag.values
    #initiate model selection class from stratified kfold module
    kf = model_selection.StratifiedKFold(n_splits = 5)
    
    # fill the new k fold column
    for f,(t_,v_) in enumerate(kf.split(X =df,y = y)):
        df.loc[v_,'kfold'] = f

    #go over the folds created
    for fold_ in range(5):
        
        #creating temporary dataframes for train and test
        train_data = df[df.kfold != fold_].reset_index(drop = True)
        test_data = df[df.kfold == fold_].reset_index(drop = True)

        #initialize the count vector tokenizer with NLTK word tokenize
        count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern = None)
        
        #fit the count vector
        count_vec.fit(train_data.TRANS_CONV_TEXT)
        pickle.dump(count_vec, open("count_vectorizer.pickle", "wb")) 
        
        #transform the training and validation data reviews
        xtrain = count_vec.transform(train_data.TRANS_CONV_TEXT)
        xtest = count_vec.transform(test_data.TRANS_CONV_TEXT)


        model = linear_model.LogisticRegression()
        model.fit(xtrain,train_data.Patient_Tag)
        
        preds = model.predict(xtest)
        roc_auc_score = metrics.roc_auc_score(test_data.Patient_Tag,preds)
        
        print(f'fold : {fold_}')
        print(f'roc_auc = {roc_auc_score}')
        print(classification_report(test_data.Patient_Tag,preds))
        print(f"Accuracy Score = {accuracy_score(test_data.Patient_Tag,preds)}")
              
        print('<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>')

fold : 0
roc_auc = 0.882246376811594
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       184
           1       0.88      0.79      0.84        48

    accuracy                           0.94       232
   macro avg       0.92      0.88      0.90       232
weighted avg       0.93      0.94      0.93       232

Accuracy Score = 0.9353448275862069
<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>
fold : 1
roc_auc = 0.8065232240437159
              precision    recall  f1-score   support

           0       0.91      0.97      0.94       183
           1       0.84      0.65      0.73        48

    accuracy                           0.90       231
   macro avg       0.88      0.81      0.83       231
weighted avg       0.90      0.90      0.90       231

Accuracy Score = 0.9004329004329005
<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>
fold : 2
roc_auc = 0.8476775956284153
              precision    recall  f1-score   support

           0       0.9

it seems that, model precision and recall values are not good when it comes to the positive class (Patient label 1). We will make more models to improve the results. 

Let's submit the first predictions

In [68]:
count_vec = pickle.load(open("./count_vectorizer.pickle", 'rb'))
test_features = count_vec.transform(df_test.TRANS_CONV_TEXT)

In [69]:
test_preds = model.predict(test_features)

In [70]:
submission_logistic_bow = pd.read_csv('../input/zsdataset/test.csv',encoding = 'ISO-8859-1')

In [72]:
submission = pd.DataFrame()
submission['Index'] = submission_logistic_bow['Index']
submission['Patient_Tag'] = test_preds
submission.to_csv('submission_logistic_bow.csv',index = False)

  We get average ROC AUC score of nearly 86 - 87 percent. We check if it increase by other algorithms

### But we can also use Naive bayes classifier, with multinomialNB from Sklearn

In [89]:
df1 = train_df.copy()
df1_test = test_df.copy()

In [91]:
from nltk.tokenize import word_tokenize
from sklearn import naive_bayes
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


if __name__ =="__main__":
    #create a new column with and fill it with -1
    df1['kfold'] = -1
    #randomize the data
    df1 = df1.sample(frac = 1).reset_index(drop =True)
    y = df1.Patient_Tag.values
    #initiate model selection class from stratified kfold module
    kf = model_selection.StratifiedKFold(n_splits = 5)
    
    # fill the new k fold column
    for f,(t_,v_) in enumerate(kf.split(X =df1,y = y)):
        df1.loc[v_,'kfold'] = f

    #go over the folds created
    for fold_ in range(5):
        
        #creating temporary dataframes for train and test
        train_data = df1[df1.kfold != fold_].reset_index(drop = True)
        test_data = df1[df1.kfold == fold_].reset_index(drop = True)

        #initialize the count vector tokenizer with NLTK word tokenize
        count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern = None)
        
        #fit the count vector
        count_vec.fit(train_data.TRANS_CONV_TEXT)
        pickle.dump(count_vec, open("count_vectorizer.pickle", "wb"))
        
        #transform the training and validation data reviews
        xtrain = count_vec.transform(train_data.TRANS_CONV_TEXT)
        xtest = count_vec.transform(test_data.TRANS_CONV_TEXT)


        model1 = naive_bayes.MultinomialNB()
        model1.fit(xtrain,train_data.Patient_Tag)
        
        preds = model1.predict(xtest)
        roc_auc_score = metrics.roc_auc_score(test_data.Patient_Tag,preds)
        
        print(f'fold : {fold_}')
        print(f'roc_auc = {roc_auc_score}')
        print(classification_report(test_data.Patient_Tag,preds))
        print(f"Accuracy Score = {accuracy_score(test_data.Patient_Tag,preds)}")
              
        print('<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>')

fold : 0
roc_auc = 0.9221014492753624
              precision    recall  f1-score   support

           0       0.99      0.89      0.93       184
           1       0.69      0.96      0.80        48

    accuracy                           0.90       232
   macro avg       0.84      0.92      0.87       232
weighted avg       0.93      0.90      0.91       232

Accuracy Score = 0.9008620689655172
<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>
fold : 1
roc_auc = 0.9014685792349728
              precision    recall  f1-score   support

           0       0.97      0.91      0.94       183
           1       0.72      0.90      0.80        48

    accuracy                           0.90       231
   macro avg       0.84      0.90      0.87       231
weighted avg       0.92      0.90      0.91       231

Accuracy Score = 0.9047619047619048
<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>
fold : 2
roc_auc = 0.9064207650273223
              precision    recall  f1-score   support

           0       0.

As we can see there is an improvement in the precision and recall values, that is good for us. Let's create submission file

In [93]:
count_vec = pickle.load(open("./count_vectorizer.pickle", 'rb'))
test_features = count_vec.transform(df1_test.TRANS_CONV_TEXT)
test_preds = model1.predict(test_features)
submission_multinomial_bow = pd.read_csv('../input/zsdataset/test.csv',encoding = 'ISO-8859-1')
submission = pd.DataFrame()
submission['Index'] = submission_logistic_bow['Index']
submission['Patient_Tag'] = test_preds
submission.to_csv('submission_multinomial_bow.csv',index = False)

By using naive bayes and CountVectorizer it reaches to nearly 92 to 93 percentage

So for baseline we will use, Naive Bayes instead of logistic regression. Now we will use TF-IDF and Multinomial and check is there an improvement ?

### TF - IDF

Okay. Let's use, TF-IDF. TF = term frequency. IDF - inverse document frequency. TF = (number of times a term 't' repeats in document / total number of terms in document)
IDF = (total number of documents / number of document with a term 't' in it)
TFIDF(t) = TF(t) * IDF(t)

Now replace Count vector tokenizer BY tf-idf in naive-bayes

In [106]:
df2 = train_df.copy()
df2_test = test_df.copy()

In [108]:
from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer



if __name__ =="__main__":
    #create a new column with and fill it with -1
    df2['kfold'] = -1
    #randomize the data
    df2 = df2.sample(frac = 1).reset_index(drop =True)
    y = df2.Patient_Tag.values
    #initiate model selection class from stratified kfold module
    kf = model_selection.StratifiedKFold(n_splits = 5)
    
    # fill the new k fold column
    for f,(t_,v_) in enumerate(kf.split(X =df2,y = y)):
        df2.loc[v_,'kfold'] = f

    #go over the folds created
    for fold_ in range(5):
        
        #creating temporary dataframes for train and test
        train_data = df2[df2.kfold != fold_].reset_index(drop = True)
        test_data = df2[df2.kfold == fold_].reset_index(drop = True)

        #initialize the count vector tokenizer with NLTK word tokenize
        tfidf_vec = TfidfVectorizer(tokenizer = word_tokenize, token_pattern = None)
        
        #fit the count vector
        tfidf_vec.fit(train_data.TRANS_CONV_TEXT)
        pickle.dump(tfidf_vec, open("tfidf_vectorizer.pickle", "wb"))
        
        #transform the training and validation data reviews
        xtrain = tfidf_vec.transform(train_data.TRANS_CONV_TEXT)
        xtest = tfidf_vec.transform(test_data.TRANS_CONV_TEXT)


        model2 = linear_model.LogisticRegression()
        model2.fit(xtrain,train_data.Patient_Tag)
        
        preds = model2.predict(xtest)
        roc_auc_score = metrics.roc_auc_score(test_data.Patient_Tag,preds)
        
        print(f'fold : {fold_}')
        print(f'roc_auc = {roc_auc_score}')
        print(f"Accuracy Score = {accuracy_score(test_data.Patient_Tag,preds)}")
        print(classification_report(test_data.Patient_Tag,preds))
        
              
        print('<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>')

fold : 0
roc_auc = 0.8147644927536233
Accuracy Score = 0.9137931034482759
              precision    recall  f1-score   support

           0       0.91      0.98      0.95       184
           1       0.91      0.65      0.76        48

    accuracy                           0.91       232
   macro avg       0.91      0.81      0.85       232
weighted avg       0.91      0.91      0.91       232

<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>
fold : 1
roc_auc = 0.8246243169398907
Accuracy Score = 0.9047619047619048
              precision    recall  f1-score   support

           0       0.92      0.96      0.94       183
           1       0.82      0.69      0.75        48

    accuracy                           0.90       231
   macro avg       0.87      0.82      0.85       231
weighted avg       0.90      0.90      0.90       231

<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>
fold : 2
roc_auc = 0.8273565573770492
Accuracy Score = 0.9090909090909091
              precision    recall  f1-sc

But it did not give better results, than bag of words :-/ 

The precision value in patient class is very low. We will save the results, but they are not good. 

In [109]:
tfidf_vec = pickle.load(open("./tfidf_vectorizer.pickle", 'rb'))
test_features = tfidf_vec.transform(df2_test.TRANS_CONV_TEXT)
test_preds = model2.predict(test_features)
submission_logistic_tfidf = pd.read_csv('../input/zsdataset/test.csv',encoding = 'ISO-8859-1')
submission = pd.DataFrame()
submission['Index'] = submission_logistic_tfidf['Index']
submission['Patient_Tag'] = test_preds
submission.to_csv('submission_logistic_tfidf.csv',index = False)

Let's try to add language models to tfidf. We can create bigram models, trigram models and incorporate them with tfidf. We will save it's result if it is good else we will leave them

In [24]:
from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer



if __name__ =="__main__":
    #create a new column with and fill it with -1
    df2['kfold'] = -1
    #randomize the data
    df2 = df2.sample(frac = 1).reset_index(drop =True)
    y = df2.Patient_Tag.values
    #initiate model selection class from stratified kfold module
    kf = model_selection.StratifiedKFold(n_splits = 5)
    
    # fill the new k fold column
    for f,(t_,v_) in enumerate(kf.split(X =df2,y = y)):
        df2.loc[v_,'kfold'] = f

    #go over the folds created
    for fold_ in range(5):
        
        #creating temporary dataframes for train and test
        train_data = df2[df2.kfold != fold_].reset_index(drop = True)
        test_data = df2[df2.kfold == fold_].reset_index(drop = True)

        #initialize the count vector tokenizer with NLTK word tokenize
        tfidf_vec = TfidfVectorizer(tokenizer = word_tokenize, token_pattern = None, ngram_range=(1,3))
        
        #fit the count vector
        tfidf_vec.fit(train_data.TRANS_CONV_TEXT)
        
        #transform the training and validation data reviews
        xtrain = count_vec.transform(train_data.TRANS_CONV_TEXT)
        xtest = count_vec.transform(test_data.TRANS_CONV_TEXT)


        model = linear_model.LogisticRegression()
        model.fit(xtrain,train_data.Patient_Tag)
        
        preds = model.predict(xtest)
        roc_auc_score = metrics.roc_auc_score(test_data.Patient_Tag,preds)
        
        print(f'fold : {fold_}')
        print(f'roc_auc = {roc_auc_score}')
        print('')

fold : 0
roc_auc = 0.8038949275362319

fold : 1
roc_auc = 0.7884221311475409

fold : 2
roc_auc = 0.8739754098360655

fold : 3
roc_auc = 0.9002732240437159

fold : 4
roc_auc = 0.8789275956284154



it improved the results but didn't beat the naive bayes

### Let's try lemmatize and check the results

In [112]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize(sentence):
    word_list = nltk.word_tokenize(sentence)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

In [113]:
df3 = train_df.copy()
df3['TRANS_CONV_TEXT']=df3['TRANS_CONV_TEXT'].apply(lambda x:lemmatize(x))

Let's check adding lemmatization improves any thing or not.

In [114]:
from nltk.tokenize import word_tokenize
from sklearn import naive_bayes 
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer



if __name__ =="__main__":
    #create a new column with and fill it with -1
    df3['kfold'] = -1
    #randomize the data
    df3 = df3.sample(frac = 1).reset_index(drop =True)
    y = df3.Patient_Tag.values
    #initiate model selection class from stratified kfold module
    kf = model_selection.StratifiedKFold(n_splits = 5)
    
    # fill the new k fold column
    for f,(t_,v_) in enumerate(kf.split(X =df3,y = y)):
        df3.loc[v_,'kfold'] = f

    #go over the folds created
    for fold_ in range(5):
        
        #creating temporary dataframes for train and test
        train_data = df3[df3.kfold != fold_].reset_index(drop = True)
        test_data = df3[df3.kfold == fold_].reset_index(drop = True)

        #initialize the count vector tokenizer with NLTK word tokenize
        tfidf_vec = TfidfVectorizer(tokenizer = word_tokenize, token_pattern = None, ngram_range=(1,3))
        
        #fit the count vector
        tfidf_vec.fit(train_data.TRANS_CONV_TEXT)
        
        #transform the training and validation data reviews
        xtrain = count_vec.transform(train_data.TRANS_CONV_TEXT)
        xtest = count_vec.transform(test_data.TRANS_CONV_TEXT)


        model3 = naive_bayes.MultinomialNB()
        model3.fit(xtrain,train_data.Patient_Tag)
        
        preds = model3.predict(xtest)
        roc_auc_score = metrics.roc_auc_score(test_data.Patient_Tag,preds)
        
        print(f'fold : {fold_}')
        print(f'roc_auc = {roc_auc_score}')
        print(f"Accuracy Score = {accuracy_score(test_data.Patient_Tag,preds)}")
        print(classification_report(test_data.Patient_Tag,preds))
        
              
        print('<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>')

fold : 0
roc_auc = 0.9012681159420289
Accuracy Score = 0.8922413793103449
              precision    recall  f1-score   support

           0       0.98      0.89      0.93       184
           1       0.68      0.92      0.78        48

    accuracy                           0.89       232
   macro avg       0.83      0.90      0.85       232
weighted avg       0.91      0.89      0.90       232

<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>
fold : 1
roc_auc = 0.910860655737705
Accuracy Score = 0.8831168831168831
              precision    recall  f1-score   support

           0       0.99      0.86      0.92       183
           1       0.65      0.96      0.77        48

    accuracy                           0.88       231
   macro avg       0.82      0.91      0.85       231
weighted avg       0.92      0.88      0.89       231

<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>
fold : 2
roc_auc = 0.8582650273224044
Accuracy Score = 0.8484848484848485
              precision    recall  f1-sco

Nope not much :-\ 

We can also use, snowball stemmering. Lemmatization and stemming both are different.  if word - fishing then stemmed - fish lemma is fishing. We can say stemming is very strict as compared to lemmatization.

### Let's impliment stemming, and check if it improves scores or not

In [18]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
def stemming(sentence):
    word_list = nltk.word_tokenize(sentence)
    stemmed_output = ' '.join([stemmer.stem(w) for w in word_list])
    return stemmed_output

In [19]:
df4 = train_df.copy()
df4['TRANS_CONV_TEXT']=df4['TRANS_CONV_TEXT'].apply(lambda x: stemming(x))
df4_test = test_df.copy()

In [22]:
from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


if __name__ =="__main__":
    #create a new column with and fill it with -1
    df4['kfold'] = -1
    #randomize the data
    df4 = df4.sample(frac = 1).reset_index(drop =True)
    y = df4.Patient_Tag.values
    #initiate model selection class from stratified kfold module
    kf = model_selection.StratifiedKFold(n_splits = 5)
    
    # fill the new k fold column
    for f,(t_,v_) in enumerate(kf.split(X =df4,y = y)):
        df4.loc[v_,'kfold'] = f

    #go over the folds created
    for fold_ in range(5):
        
        #creating temporary dataframes for train and test
        train_data = df4[df4.kfold != fold_].reset_index(drop = True)
        test_data = df4[df4.kfold == fold_].reset_index(drop = True)

        #initialize the count vector tokenizer with NLTK word tokenize
        count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern = None, ngram_range=(1,3))
        
        #fit the count vector
        count_vec.fit(train_data.TRANS_CONV_TEXT)
        
        pickle.dump(count_vec, open("count_vectorizer.pickle", "wb"))
        #transform the training and validation data reviews
        xtrain = count_vec.transform(train_data.TRANS_CONV_TEXT)
        xtest = count_vec.transform(test_data.TRANS_CONV_TEXT)

        
        model5 = linear_model.LogisticRegression()
        model5.fit(xtrain,train_data.Patient_Tag)
        
        preds = model5.predict(xtest)
        roc_auc_score = metrics.roc_auc_score(test_data.Patient_Tag,preds)
        
        print(f'fold : {fold_}')
        print(f'roc_auc = {roc_auc_score}')
        print(f"Accuracy Score = {accuracy_score(test_data.Patient_Tag,preds)}")
        print(classification_report(test_data.Patient_Tag,preds))
        
              
        print('<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>')

fold : 0
roc_auc = 0.8355978260869565
Accuracy Score = 0.9224137931034483
              precision    recall  f1-score   support

           0       0.92      0.98      0.95       184
           1       0.92      0.69      0.79        48

    accuracy                           0.92       232
   macro avg       0.92      0.84      0.87       232
weighted avg       0.92      0.92      0.92       232

<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>
fold : 1
roc_auc = 0.8191598360655736
Accuracy Score = 0.8961038961038961
              precision    recall  f1-score   support

           0       0.92      0.95      0.94       183
           1       0.79      0.69      0.73        48

    accuracy                           0.90       231
   macro avg       0.85      0.82      0.83       231
weighted avg       0.89      0.90      0.89       231

<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>
fold : 2
roc_auc = 0.8218920765027322
Accuracy Score = 0.9004329004329005
              precision    recall  f1-sc

Yes, it improved our results. We will implement it in our base models

In [23]:
df4_test['TRANS_CONV_TEXT']=df4_test['TRANS_CONV_TEXT'].apply(lambda x: stemming(x))
count_vec = pickle.load(open("./count_vectorizer.pickle", 'rb'))
test_features = count_vec.transform(df4_test.TRANS_CONV_TEXT)
test_preds = model5.predict(test_features)
submission_stem_count = pd.read_csv('../input/zsdataset/test.csv',encoding = 'ISO-8859-1')
submission = pd.DataFrame()
submission['Index'] = submission_stem_count['Index']
submission['Patient_Tag'] = test_preds
submission.to_csv('submission_stem_count.csv',index = False)

But we can also provide word embeddings. In above all methods, we convered word tokens in numbers. So if there are N unique tokens then they can be represented by integers ranges from 0 to N - 1. We can convert those numbers in to vectors. 

In [31]:
df4 = train_df.copy()
df4['TRANS_CONV_TEXT']=df4['TRANS_CONV_TEXT'].apply(lambda x: stemming(x))

In [32]:
import numpy as np
import io
from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer

def sentence_to_vec(s,embedding_dict, stop_words, tokenizer):
    words = str(s).lower()
    words = re.sub(r"[^a-z']+", ' ', words)
    words = tokenizer(words)
    #remove the stop words
    words = [w for w in words if not w in stop_words]
    M = []
    for w in words:
        #for each word fetch embedding from the dictionary and append the list of embeddings
        if w in embedding_dict:
            M.append(embedding_dict[w])
    #if we dont have vectors then retuen zeros
    if len(M) == 0:
        return np.zeros(300)
    #convert list of embeddings to array
    M = np.array(M)
    #calculate the sum over axis = 0
    v = M.sum(axis = 0)
    #return normalized vector
    return v/np.sqrt((v**2).sum())

#for embeddings we need to load vectors
def load_vectors(fname):
    fin = io.open(fname,'r',encoding='utf-8',newline = '\n', errors = 'ignore')
    n,d = map(int,fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.strip().split(' ')
        data[tokens[0]] = list(map(float,tokens[1:]))
    return data

if __name__ =="__main__":
    #create a new column with and fill it with -1
    df4['kfold'] = -1
    #randomize the data
    df4 = df4.sample(frac = 1).reset_index(drop =True)
    # load embeddings into memory
    print('loading embeddings')
    embeddings = load_vectors('../input/fasttext-wikinews/wiki-news-300d-1M.vec')
    vectors = []
    for TRANS_CONV_TEXT in df4.TRANS_CONV_TEXT.values:
        vectors.append(sentence_to_vec(s = TRANS_CONV_TEXT, embedding_dict = embeddings, stop_words = [], tokenizer = word_tokenize))
    vectors = np.array(vectors)
    
    y = df4.Patient_Tag.values
    #initiate model selection class from stratified kfold module
    kf = model_selection.StratifiedKFold(n_splits = 5)
    
    # fill the new k fold column
    for f,(t_,v_) in enumerate(kf.split(X = vectors,y = y)):
        print(f'Training fold : {f}')
        xtrain = vectors[t_,:]
        ytrain = y[t_]
        
        xtest = vectors[v_,:]
        ytest = y[v_]
        
        model = linear_model.LogisticRegression()
        model.fit(xtrain, ytrain)
        
        preds = model.predict(xtest)
        
        roc_auc_score = metrics.roc_auc_score(ytest,preds)
        
        print(f'fold : {fold_}')
        print(f'roc_auc = {roc_auc_score}')
        print('')


loading embeddings
Training fold : 0
fold : 4
roc_auc = 0.5828804347826086

Training fold : 1
fold : 4
roc_auc = 0.6612021857923497

Training fold : 2
fold : 4
roc_auc = 0.7028688524590164

Training fold : 3
fold : 4
roc_auc = 0.637636612021858

Training fold : 4
fold : 4
roc_auc = 0.7023565573770492



I used, fasttext wikinews word vectors, although, other word vectors available, can be used as per convience. But due to memory constraints I tried with this one. Which did not give us good results

Now deep learning approach.

We will make a simple LSTM model and build a model. We will directly use, stemming as it gave us good results previously. And won't take 5 folds now. We will simply divide dataset into train and test where 80% are traing data and 20% as testing data and split it with train test split of sklearn. 